# Identify the 3 top matching chunks using LlamaIndex

LlamaIndex:
- Github: https://github.com/run-llama/llama_index
- Documentation: https://docs.llamaindex.ai/en/latest/

In [1]:
import pandas as pd

# read the xlsx data into a pandas dataframe
df = pd.read_excel('../data/ReferenceErrorDetection_data.xlsx')

In [2]:
df.head()

,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Statement with Citation,Reference Article ID,Reference Article DOI,Reference Article Title,Reference Article Abstract,Reference Article PDF Available,Reference Article Retracted,Reference Article Downloaded,Label,Explanation
0,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Others have aimed to reduce irreversibility or...,r001,10.1155/2021/2087027,A Fault Analysis Method for Three-Phase Induct...,The fault prediction and abductive fault diagn...,Yes,No,Yes,Unsubstantiate,Irrelevant
1,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Some researchers have also studied various hea...,r002,10.1016/j.physa.2018.12.031,Develop 24 dissimilar ANNs by suitable archite...,The artificial neural network optimization met...,Yes,No,Yes,Unsubstantiate,Irrelevant
2,PubPeer,c002,10.1155/2022/4601350,Oxidative Potential and Nanoantioxidant Activi...,Yes,Yes,Chemistry,The relative content of total flavonoids in th...,r003,10.1088/1742-6596/1937/1/012038,Lipid Data Acquisition for devices Treatment o...,"Recently, the widespread deployment of smart p...",Yes,No,Yes,Unsubstantiate,Irrelevant
3,PubPeer,c003,10.1155/2022/2408685,The Choice of Anesthetic Drugs in Outpatient H...,Yes,Yes,Medicine,Research has shown that remimazolam tosylate e...,r004,10.1186/s12871-018-0543-3,"Effect of propofol on breast cancer cell, the ...",Breast cancer is the second leading cause of c...,Yes,No,Yes,Unsubstantiate,Irrelevant
4,PubPeer,c004,10.1155/2022/4783847,A Fault-Tolerant Structure for Nano-Power Comm...,Yes,Yes,Engineering,if the efficiency of the routing algorithm is ...,r005,10.36410/jcpr.2022.23.3.312,Analysis and research hotspots of ceramic mate...,"From the perspective of scientometrics, comb t...",Yes,No,Yes,Unsubstantiate,Irrelevant


## Get text from reference article

In [58]:
grobid_model = "full_model_texts"
extension = "txt"

In [35]:
import glob

def get_file_path(reference_article_id):
    # Construct the file path pattern using the Reference Article ID of the first entry
    file_pattern = f"../data/extractions/{grobid_model}/{reference_article_id}*.{extension}"

    # Find the file that matches the pattern
    file_list = glob.glob(file_pattern)
    if file_list:
        file_path = file_list[0]
        return file_path
    else: 
        print("No matching file found.")
        return None

In [36]:
import xml.etree.ElementTree as ET

def get_reference_text(reference_article_id):
    global extension
    
    # Get the file path
    file_path = get_file_path(reference_article_id)
    
    if file_path:
        if extension == "txt":
            # Read the text file
            with open(file_path, 'r') as file:
                reference_text = file.read()
            return reference_text

        elif extension == "xml":
            # Parse the XML file
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Extract the text content from the XML file
            reference_text = ''.join(root.itertext())
            return reference_text

## Set OpenAI key

In [37]:
# Read the content of open_ai_key.txt into a variable
with open('../open_ai_key.txt', 'r') as file:
    open_ai_key = file.read().strip()

## Setting up vector index

### Reloading or generating index

In [59]:
embedding = "te3s" # / "te3s"

In [60]:
if embedding == "te3l":
    model_embeddings = "text-embedding-3-large"
elif embedding == "te3s":
    model_embeddings = "text-embedding-3-small"

In [61]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model=model_embeddings, api_key=open_ai_key)

In [62]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.ingestion import IngestionPipeline

def create_index(reference_text, chunk_size, chunk_overlap):
    # create the pipeline with transformations
    pipeline = IngestionPipeline(
        transformations=[
            SentenceSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap),
            OpenAIEmbedding(model=model_embeddings, api_key=open_ai_key)
        ]
    )

    # run the pipeline
    nodes = pipeline.run(documents=[Document(text=reference_text)])
    index = VectorStoreIndex(nodes)
    return index

In [63]:
from llama_index.core.node_parser import TokenTextSplitter

def create_chunks(text, chunk_size=256, chunk_overlap=20):
    token_text_splitter = TokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = token_text_splitter.split_text(text)
    return chunks

In [64]:
def create_index_with_chunking_first(reference_text, chunk_size, chunk_overlap):
    reference_chunks = create_chunks(reference_text, chunk_size, chunk_overlap)
    documents = [Document(text=chunk) for chunk in reference_chunks]
    index = VectorStoreIndex.from_documents(documents)
    return index

In [65]:
from llama_index.core import StorageContext, load_index_from_storage

def load_or_create_index(article_id, reference_text, no_prev_chunking, chunk_size=256, chunk_overlap=20):
    index_path = f"../data/vector_indices/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/{article_id}/"
    index = None

    try:
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        index = load_index_from_storage(storage_context)
        print(article_id + ": Loaded existing index.")
    except Exception as e:
        print(e)
        print(article_id + ": Creating a new index.")
        try: 
            if no_prev_chunking:
                index = create_index(reference_text, chunk_size, chunk_overlap)
            else:
                index = create_index_with_chunking_first(reference_text, chunk_size, chunk_overlap)
            index.storage_context.persist(persist_dir=index_path)
        except Exception as e:
            print(e)
            print(article_id + ": Failed to create index.")
            print(reference_text)
    return index

### Create Indices for all reference articles

In [66]:
ids_to_index = ["r071", "r075", "r147"]
no_prev_chunking = False

for _, row in df.iterrows():
    if row['Reference Article Downloaded'] == 'Yes':
        reference_article_id = row['Reference Article ID']
        if reference_article_id and (len(ids_to_index) == 0 or reference_article_id in ids_to_index):
            reference_text = get_reference_text(reference_article_id)
            index = load_or_create_index(reference_article_id, reference_text, no_prev_chunking)

[Errno 2] No such file or directory: '/home/ibelter/master_thesis/citation-verification/notebooks/../data/vector_indices/te3s/full_model_texts/r071/docstore.json'
r071: Creating a new index.
[Errno 2] No such file or directory: '/home/ibelter/master_thesis/citation-verification/notebooks/../data/vector_indices/te3s/full_model_texts/r075/docstore.json'
r075: Creating a new index.
[Errno 2] No such file or directory: '/home/ibelter/master_thesis/citation-verification/notebooks/../data/vector_indices/te3s/full_model_texts/r147/docstore.json'
r147: Creating a new index.


## Identifying top 3 chunks

In [67]:
from llama_index.core.retrievers import VectorIndexRetriever

def get_top_k_similar_chunks(statement, index, k=3):
    retriever = VectorIndexRetriever(
        index=index,
        similarity_top_k=k,
    )
    retrieved_nodes = retriever.retrieve(statement)
    return retrieved_nodes

In [68]:
import json
import os

def save_similar_chunks(doc_ids, reference_id):
    file_path = f"../data/similar_chunks/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/{reference_id}.json"
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as file:
        json.dump(doc_ids, file)

def load_similar_chunks(reference_id):
    file_path = f"../data/similar_chunks/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/{reference_id}.json"
    with open(file_path, 'r') as file:
        doc_ids = json.load(file)
    return doc_ids

### Save the document ids and text contents of the top k chunks of all reference articles to the df

In [69]:
def get_doc_ids(response):
    doc_ids = []
    for node in response:
        doc_ids.append(node.dict()['node']['id_'])
    return doc_ids

In [70]:
def save_top_k_chunk_ids(df, no_prev_chunking, k=3):
    for _, row in df.iterrows():
        if row['Reference Article Downloaded'] == 'Yes':
            reference_article_id = row['Reference Article ID']
            print(f"------ Starting {reference_article_id} ------")
            
            # Try to load similar chunks first
            try:
                doc_ids = load_similar_chunks(reference_article_id)
                print("Loaded similar chunks successfully.")
            except FileNotFoundError:
                # Load reference text and create chunks
                reference_text = get_reference_text(reference_article_id)
                
                # Load or create index
                index = load_or_create_index(reference_article_id, reference_text, no_prev_chunking)
                
                # Get the statement and retrieve top chunks
                statement = row["Statement with Citation"]
                print("Receiving top chunks")

                try:
                    response = get_top_k_similar_chunks(statement, index, k)
                    doc_ids = get_doc_ids(response)
                    
                    # Save the top chunks
                    print("Saving top chunks")
                    save_similar_chunks(doc_ids, reference_article_id)
                except Exception as e:
                    print(e)
                    print("Failed to get top chunks.")
            print("")

In [71]:
no_prev_chunking = False
save_top_k_chunk_ids(df, no_prev_chunking)

------ Starting r001 ------
Loaded similar chunks successfully.

------ Starting r002 ------
Loaded similar chunks successfully.

------ Starting r003 ------
Loaded similar chunks successfully.

------ Starting r004 ------
Loaded similar chunks successfully.

------ Starting r005 ------
Loaded similar chunks successfully.

------ Starting r006 ------
Loaded similar chunks successfully.

------ Starting r007 ------
Loaded similar chunks successfully.

------ Starting r008 ------
Loaded similar chunks successfully.

------ Starting r009 ------
Loaded similar chunks successfully.

------ Starting r010 ------
Loaded similar chunks successfully.

------ Starting r011 ------
Loaded similar chunks successfully.

------ Starting r012 ------
Loaded similar chunks successfully.

------ Starting r013 ------
Loaded similar chunks successfully.

------ Starting r013 ------
Loaded similar chunks successfully.

------ Starting r014 ------
Loaded similar chunks successfully.

------ Starting r015 ----

In [77]:
import pandas as pd

output_dir = f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/"
df2 = pd.read_pickle(os.path.join(output_dir, f"ReferenceErrorDetection_data_with_chunk_info.pkl"))

In [82]:
def add_top_k_chunk_ids_and_texts_to_df(df, k=3):
    doc_ids_list = []
    doc_texts_list = []
    for _, row in df.iterrows():
        if row['Reference Article Downloaded'] == 'Yes':
            reference_article_id = row['Reference Article ID']

            if len(ids_to_index) != 0 and reference_article_id not in ids_to_index:
                doc_ids_list.append(row[f'Top_{k}_Chunk_IDs'])
                doc_texts_list.append(row[f'Top_{k}_Chunk_Texts'])
                continue
            
            print(f"------ Starting {reference_article_id} ------")

            # load index
            index_path = f"../data/vector_indices/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/{reference_article_id}/"
            storage_context = StorageContext.from_defaults(persist_dir=index_path)
            index = load_index_from_storage(storage_context)

            # load similar chunks
            doc_ids = load_similar_chunks(reference_article_id)
            doc_texts = [index.docstore.docs[doc_id].text for doc_id in doc_ids]

            # add to lists
            doc_ids_list.append(doc_ids)
            doc_texts_list.append(doc_texts)
        else:
            doc_ids_list.append(None)
            doc_texts_list.append(None)
    
    df[f'Top_{k}_Chunk_IDs'] = doc_ids_list
    df[f'Top_{k}_Chunk_Texts'] = doc_texts_list
    return df

In [83]:
ids_to_index = ["r071", "r075", "r147"]
df2 = add_top_k_chunk_ids_and_texts_to_df(df2)

------ Starting r071 ------
------ Starting r075 ------
------ Starting r147 ------


In [84]:
df2.head()

,Source,Citing Article ID,Citing Article DOI,Citing Article Title,Citing Article Retracted,Citing Article Downloaded,Domain,Statement with Citation,Reference Article ID,Reference Article DOI,Reference Article Title,Reference Article Abstract,Reference Article PDF Available,Reference Article Retracted,Reference Article Downloaded,Label,Explanation,Top_3_Chunk_IDs,Top_3_Chunk_Texts
0,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Others have aimed to reduce irreversibility or...,r001,10.1155/2021/2087027,A Fault Analysis Method for Three-Phase Induct...,The fault prediction and abductive fault diagn...,Yes,No,Yes,Unsubstantiate,Irrelevant,"[00533ca6-92f8-4649-9bba-4d0c2616b39a, 6c75579...",[they cannot effectively diagnose multiple fau...
1,PubPeer,c001,10.1016/j.est.2021.103553,Heating a residential building using the heat ...,Yes,Yes,Engineering,Some researchers have also studied various hea...,r002,10.1016/j.physa.2018.12.031,Develop 24 dissimilar ANNs by suitable archite...,The artificial neural network optimization met...,Yes,No,Yes,Unsubstantiate,Irrelevant,"[a359aaa3-b2cf-47dd-8b5b-414acd5c38ec, 3a469c7...",[properties. The evaluations of nanofluid ther...
2,PubPeer,c002,10.1155/2022/4601350,Oxidative Potential and Nanoantioxidant Activi...,Yes,Yes,Chemistry,The relative content of total flavonoids in th...,r003,10.1088/1742-6596/1937/1/012038,Lipid Data Acquisition for devices Treatment o...,"Recently, the widespread deployment of smart p...",Yes,No,Yes,Unsubstantiate,Irrelevant,"[2af26130-5f29-4b0f-b47f-019d9b2d66f1, 8b62706...",[AND RESULT\nPhotochemical blood performance o...
3,PubPeer,c003,10.1155/2022/2408685,The Choice of Anesthetic Drugs in Outpatient H...,Yes,Yes,Medicine,Research has shown that remimazolam tosylate e...,r004,10.1186/s12871-018-0543-3,"Effect of propofol on breast cancer cell, the ...",Breast cancer is the second leading cause of c...,Yes,No,Yes,Unsubstantiate,Irrelevant,"[4947cbac-d6d7-4f50-9c75-4c101035c923, f9560dc...","[0.001) at 1 year, and 2% (-2 to 6%, non-signi..."
4,PubPeer,c004,10.1155/2022/4783847,A Fault-Tolerant Structure for Nano-Power Comm...,Yes,Yes,Engineering,if the efficiency of the routing algorithm is ...,r005,10.36410/jcpr.2022.23.3.312,Analysis and research hotspots of ceramic mate...,"From the perspective of scientometrics, comb t...",Yes,No,Yes,Unsubstantiate,Irrelevant,"[27f6d293-236a-486b-b3e9-6970081048b0, afd3372...",[space can reflect the intermediary status of ...


In [85]:
import os

# Ensure the directory exists
output_dir = f"../data/dfs/{embedding}{'_no_prev_chunking' if no_prev_chunking else ''}/{grobid_model}/"
os.makedirs(output_dir, exist_ok=True)

# Save the DataFrame to a pickle file
df2.to_pickle(os.path.join(output_dir, f"ReferenceErrorDetection_data_with_chunk_info.pkl"))